# Data cleaning
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for using a language model to conduct a data cleaning task. In a series of steps we use EDSL to prompt a language model to generate appropriate sense checks for a dataset and then run the sense checks in the form of a survey about the data, returning a new dataset consisting of the data failing the checks.

[EDSL is an open-source library](https://github.com/expectedparrot/edsl) for simulating surveys, experiments and other research with AI agents and large language models. 
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Example data
Here we construct a dataset for our exercise: a random list of ages between 22 and 85 with some bad values mixed in.
Our goal is to identify them:

In [1]:
ages = [84, 62, 79, 57, 59, 55, 68, 66, 47, 54, 76, 33, 74, 56, 47, 24, 23, 38, 38, 54, 51, 84, 71, 
        46, 38, 26, 50, 56, 62, 39, 31, 52, 69, 84, 69, 48, 48, 23, 65, 54, 78, 51, 69, 77, 75, 76, 
        26, 44, 61, 32, 70, 24, 74, 22, 32, 24, 80, 65, 36, 42, 84, 66, 40, 85, 28, 22, 67, 25, 70, 
        77, 53, 69, 64, 27, 61, 68, 68, 78, 0.99, 83, 58, 33, 46, 43, 50, 85, 28, 82, 50, 61, 66, 32, 
        45, 70, 56, 50, 43, 30, 43, 55, 33, 72, 43, 43, -5, 32, 43, 45, 67, 84, 37, 63, 52, 53, 58, 
        79, 79, 80, 62, 75, 57, 60, 39, 79, 49, 60, 60, 37, 45, 36, 1050, 73, 70, 56, 39, 58, 69, 77, 
        68, 84, 78, 48, 31, 74, 27, 55, 56, 66, 35, 39, 57, 47, 29, 24, 47, 60, 43, 37, 84, 64, 28, 
        22, 37, 71, 77, 76, 84, 63, 76, 58, 41, 72, 22, 63, 78, 49, 82, 69, "old", 37, 27, 29, 54, 83, 
        80, 74, 48, 76, 49, 26, 38, 35, 36, 25, 23, 71, 33, 39, 40, 35, 85, 24, 57, 85, 63, 53, 62, 
        47, 69, 76, 71, 48, 62, 23, 25, 84, 32, 63, 75, 31, 25, 50, 85, 36, 58, 85, 34, 62, 43, 2, 
        50, 83, 44, 73, 81, 44, 43, 82, 84, 30, 24, 63, 63, 59, 46, 30, 62, 25, 52, 23, 100, 1.3, 3]

## Quick question
With a small dataset, we may be able to design the entire task as a single question where we prompt a model to review all the data at once and flag bad data:

In [2]:
from edsl import QuestionList, Scenario

q = QuestionList(
    question_name = "bad_ages",
    question_text = """
    Review the following list of observations of human ages 
    and return a list of all the problematic ages: {{ ages }}
    """ 
)

s = Scenario({"ages":ages})

results = q.by(s).run()

results.select("bad_ages", "bad_ages_comment").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                          ┃ comment                                                                       ┃
┃ .bad_ages                       ┃ .bad_ages_comment                                                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ [0.99, -5, 1050, 'old', 1.3, 3] │ # The problematic ages are those that are non-integer values, negative,       │
│                                 │ extremely high (unrealistic for human age), or non-numeric (like "old").      │
└─────────────────────────────────┴───────────────────────────────────────────────────────────────────────────────┘

This approach may be feasible for a small dataset that is easily checked.
For larger datasets, we may encounter problems with input token limits, a model's ability to accurately check a large volume of data at once, and responses that are not usefully formatted.

Below we demonstrate some ways of approaching the task in an iterative manner instead.

## Constructing a question
We start by creating a question to prompt a model to draft sense check questions for our data. 
EDSL comes with a [variety of question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the desired form of the response (multiple choice, free text, etc.). 
Here we use `QuestionList` in order to prompt the model to format its response as a list. 
We use a `{{ placeholder }}` for content that we will add to the question when we run it (a description of the data and a sample); this allows us to re-use the question with other contexts as desired:

In [3]:
from edsl import QuestionList

q1 = QuestionList(
    question_name = "sense_check_questions",
    question_text = """
    You are being asked to suggest a list of sense checks for a dataset consisting of {{ data_description }}.
    Here is a sample of the data: {{ sample_data }}.
    Format the sense checks as a list of questions to be answered about each item in the dataset individually,
    using '<data>' as a placeholder for an item being reviewed in each question text.
    """,
    max_list_items = 4 # optional
)

## Adding context to the question
Next we create `Scenario` objects representing the content that we want to add to the question when we run it.
Here we create a single scenario for our example data:

In [4]:
import random

sample_data = random.sample(ages, 10)

In [5]:
from edsl import Scenario

s = Scenario({
    "data_description": "a list of human ages (in years)",
    "sample_data": sample_data
})

## Running the question
We administer the question to a model by adding the scenarios and calling the `run` method.
This generates a formatted dataset of `Results` that we can access with [built-in methods for analysis](https://docs.expectedparrot.com/en/latest/results.html).
Here we inspect the answer:

In [6]:
results = q1.by(s).run()

In [7]:
results.select("sense_check_questions").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                                                                          ┃
┃ .sense_check_questions                                                                                          ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ['Is <data> a non-negative number?', 'Is <data> less than or equal to 120?', 'Is <data> an integer?', 'Is       │
│ <data> within a plausible human age range?']                                                                    │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

## Conducting the task
Next we want a model to answer each sense check question about each piece of data in the dataset.
This can be done by using the sense check questions as scenarios of a new question explaining the task.
We can use `QuestionYesNo` to easily filter the responses:

In [8]:
from edsl import QuestionYesNo

q2 = QuestionYesNo(
    question_name = "check_data",
    question_text = """
    You are being asked to sense check a dataset consisting of {{ data_description }}.
    Consider the following item in the dataset: {{ age }}
    {{ sense_check_question }}
    """
)

We need to create a new set of scenarios for the question.
We use `ScenarioList` objects to create all the combinations of values to add to the question (learn more about [constructing scenarios from different data sources](https://docs.expectedparrot.com/en/latest/scenarios.html)):

In [9]:
from edsl import ScenarioList

sl = ScenarioList(
    Scenario({
        "data_description": "a list of human ages (in years)",
        "sample_data": sample_data,
        "age": age,
        "sense_check_question": sense_check_question
    }) for age in ages for sense_check_question in results.select("sense_check_questions").to_list()[0]
)

We can inspect the scenarios that we created:

In [10]:
sl.sample(3)

ScenarioList([Scenario({'data_description': 'a list of human ages (in years)', 'sample_data': [79, 62, 62, 46, 69, 43, 79, 56, 24, 54], 'age': 54, 'sense_check_question': 'Is <data> an integer?'}), Scenario({'data_description': 'a list of human ages (in years)', 'sample_data': [79, 62, 62, 46, 69, 43, 79, 56, 24, 54], 'age': 54, 'sense_check_question': 'Is <data> a non-negative number?'}), Scenario({'data_description': 'a list of human ages (in years)', 'sample_data': [79, 62, 62, 46, 69, 43, 79, 56, 24, 54], 'age': 49, 'sense_check_question': 'Is <data> a non-negative number?'})])

Same as with a single scenario, we add all the scenarios to the question at once when we run it:

In [11]:
results = q2.by(sl).run()

We can filter, sort, select and print any components of the results that are generated:

In [12]:
(
    results
    .filter("check_data == 'No'")
    .sort_by("sense_check_question")
    .select("sense_check_question", "age").print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ scenario                                      ┃ scenario ┃
┃ .sense_check_question                         ┃ .age     ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━┩
│ Is <data> a non-negative number?              │ old      │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> a non-negative number?              │ -5       │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> a non-negative number?              │ 1.3      │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> an integer?                         │ 1.3      │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> an integer?                         │ 0.99     │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> an integer?                         │ old      │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> less than or equal to 120?          │ -5       │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> less than or equal to 120?          │ 1050     │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> less than or equal to 120?          │ old      │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> less than or equal to 120?          │ 1.3      │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> within a plausible human age range? │ old      │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> within a plausible human age range? │ 0.99     │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> within a plausible human age range? │ -5       │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> within a plausible human age range? │ 1050     │
├───────────────────────────────────────────────┼──────────┤
│ Is <data> within a plausible human age range? │ 1.3      │
└───────────────────────────────────────────────┴──────────┘

## Further exploration
This notebook can be readily edited and expanded for other data cleaning and data labeling purposes, or to add personas for AI agents answering the questions with relevant background and expertise. [Learn more about using AI agents for your EDSL surveys](https://docs.expectedparrot.com/en/latest/agents.html). 

Please see our [documentation page](https://docs.expectedparrot.com/) for examples of other methods and use cases and let us know if you have any questions!

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post this notebook:

In [13]:
from edsl import Notebook

In [14]:
n = Notebook(path = "data_cleaning.ipynb")

In [15]:
n.push(description = "Example code for data cleaning", visibility = "public")

{'description': 'Example code for data cleaning',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/9716dd0c-d587-4a95-817c-624618b4c429',
 'uuid': '9716dd0c-d587-4a95-817c-624618b4c429',
 'version': '0.1.33',
 'visibility': 'public'}

To update an object at the Coop:

In [16]:
n = Notebook(path = "data_cleaning.ipynb") # resave

In [17]:
n.patch(uuid = "9716dd0c-d587-4a95-817c-624618b4c429", value = n)

{'status': 'success'}